
# Access MySQL Database Server using Python

## Objectives

After completing this lab you will be able to:

-   Create a table
-   Insert data into the table
-   Query data from the table
-   Retrieve the result set into a pandas dataframe
-   Close the database connection


**Notice:** Please follow the previous notebook "MySQL_database_connection_with_Python" instructions to create a database service instance of MySQL.

##  Import `mysql.connector` Python library

`mysql.connector` [API ](https://pypi.org/project/mysql-connector-python/) provides a variety of useful Python functions for accessing and manipulating data in MySQL database, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.

We import the mysql.connector library into our Python Application


In [1]:
import mysql.connector as mysql

When the command above completes, `mysql.connector` library is loaded in your notebook. 

## Identify the database connection credentials

Connecting to MySQL database requires the following information:

-   User
-   Password
-   Host 
-   Database

**Notice:** To obtain credentials please refer to the instructions [how to connect to MySQL database server using Python
](https://gist.github.com/ttadesusi/1a6ea219fc0c5549028710c06b82dbd8) 

Now enter your database credentials below

In [2]:
# PLEASE MODIFY THE INPUTS - According to your mysql connection credentials and RUN it with Shift + Enter

from dotenv import load_dotenv
load_dotenv() 

import os 

user_var = os.environ.get('mysql_username')      # e.g. 'root'
password_var = os.environ.get('mysql_password')  # e.g. 'sample-password'
host_var = os.environ.get('mysql_hostname')      # e.g. 'localhost'
database_var = 'ibm_sql_lab'                     # e.g  'world' no need to store database name in user's environment variable

## Create the database connection

MySQL.Connector provides standards-based drivers for JDBC, ODBC, and .Net enabling developers to build database applications in their language of choice. 

Create the database connection

In [3]:
# DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter

from mysql.connector import errorcode

try:
    db_conn = mysql.connect(
                            user = user_var,         
                            password = password_var,  
                            host = host_var,         
                            database = database_var  
                            )
    print("Connected to database: " + str(database_var))
    
except mysql.Error as err: 
    
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
            
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
            
    else:
        print(err)

Connected to database: ibm_sql_lab


## Create an instance of 'cursor' class to execute the 'SQL' statements

Database cursor is a control structure that enables traversal over the records in a database. Cursors are used to process individual rows returned by database system queries. Cursors enable manipulation of whole result sets at once. 

Now use database cursors to execute and fetch all databases and tables in ibm sql lab database.

In [4]:
# DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter

# To create an instance
cursor = db_conn.cursor()          

# To compile the SQL statement "SHOW DATABASES"
cursor.execute("SHOW DATABASES")   

# To fetch a list of all databases present
databases = cursor.fetchall() 

# Show all databases one by one 
for database in databases:
    print(database)

('hr_database',)
('ibm_sql_lab',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [5]:
# DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter

# To compile the SQL statement "SHOW TABLES"
cursor.execute("SHOW TABLES")      

# To fetch a list of all tables present
tables = cursor.fetchall()        

# Show all tables one by one 
for table in tables:
    print(table)

('instructor',)
('petsale',)


## Create a table in the database

In this step we will create a table in the database with following details:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


In [6]:
# Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = cursor.execute(dropQuery)

Dont worry if you get this error:

If you see an error similar to the following, indicating an unknown table 'ibm sql lab.instructor', that's okay. It just implies that the INSTRUCTOR table does not exist in the table - which would be the case if you had not created it previously.

In [7]:
# Construct the Create Table DDL statement
createQuery = ( "CREATE TABLE instructor"
                                        "(ID INTEGER PRIMARY KEY NOT NULL,"
                                        "FNAME VARCHAR(20),"
                                        "LNAME VARCHAR(20),"
                                        "CITY VARCHAR(20)," 
                                        "CCODE CHARACTER(2))"
              )

# Now fill in the name of the method and execute the statement
createStmt = cursor.execute(createQuery)

##  Insert data into the table

In this step we will insert some rows of data into the table. 

The INSTRUCTOR table we created in the previous step contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

We will start by inserting just the first row of data, i.e. for instructor Rav Ahuja 


In [8]:
# Construct the insert query 
insertQuery = ("INSERT INTO instructor (id, fname, lname, city, ccode)"
                                "VALUES(1, 'Rav', 'Ahuja', 'Toronto', 'CA')"
              )

#execute the insert statement
insertStmt = cursor.execute(insertQuery)

# commit the inserted values to the database
db_conn.commit()

Another method that can be used to insert values into the table is by having a varable name called values containing VALUES data and running cursor.execute(query, values).

To insert more than one row, use cursor.executemany(query,values)

Now use cursor.executemany to insert the remaining two row.

In [9]:
# Insert statement that inerts the remaining two rows of data
insertQuery2 = ("INSERT INTO instructor VALUES (%s, %s, %s, %s, %s)")
values = [(2, 'Raul', 'Chong', 'Toronto', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')]

#execute the statement
insertStmt2 = cursor.executemany(insertQuery2, values)

# commit the inserted values to the database
db_conn.commit()

## Query data in the table

In this step we will retrieve data we inserted into the INSTRUCTOR table. 


In [10]:
# Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "SELECT * FROM instructor"

# Execute the statement
selectStmt = cursor.execute(selectQuery)

# Fetch the rows in instructor table
instructor_table = cursor.fetchall()
print(instructor_table)

print() # Insert space between the two outputs

# Fetch the first rows in instructor table
print(instructor_table[0])

[(1, 'Rav', 'Ahuja', 'Toronto', 'CA'), (2, 'Raul', 'Chong', 'Toronto', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')]

(1, 'Rav', 'Ahuja', 'Toronto', 'CA')


Bonus: now write and execute an update statement that changes the Rav's CITY to MOOSETOWN 


## Update data in the table

Write and execute an update statement that changes the Rav's CITY to MOOSETOWN 

In [11]:
# Construct the update statement
updateQuery = "UPDATE instructor SET city = 'Moosetown' WHERE fname = 'Rav'"

# Execute the update statement
updateStmt = cursor.execute(updateQuery)

# Commit the the update data to the database
db_conn.commit()

# Fetch and print result of the update statement
selectStmt = cursor.execute(selectQuery)
instructor_table = cursor.fetchall()
print(instructor_table[0])

(1, 'Rav', 'Ahuja', 'Moosetown', 'CA')


## Task 7: Retrieve data into Pandas

In this step we will retrieve the contents of the INSTRUCTOR table into a Pandas dataframe


In [12]:
import pandas as pd 

In [13]:
# Query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

# Retrieve the query results into a pandas dataframe
instructor_df = pd.read_sql(selectQuery, db_conn)

#print the entire data frame
instructor_df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Moosetown,CA
1,2,Raul,Chong,Toronto,CA
2,3,Hima,Vasudevan,Chicago,US


In [14]:
#print just the LNAME for first row in the pandas data frame
print(instructor_df.LNAME[0])

Ahuja


Once the data is in a Pandas dataframe, you can do the typical pandas operations on it. 

For example you can use the shape method to see how many rows and columns are in the dataframe


In [15]:
instructor_df.shape

(3, 5)

## Task 8: Close the Connection

We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources.


In [16]:
db_conn.close

<bound method CMySQLConnection.close of <mysql.connector.connection_cext.CMySQLConnection object at 0x000002DD9112C6C8>>

## Author

<a href="https://www.linkedin.com/in/ttadesusi">Temitope Adesusi</a>
 

## Reference

<a href="https://www.coursera.org/professional-certificates/ibm-data-science?">IBM Data Science</a> 

<a href="https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html">MySQL Documentation</a> 

<a href="https://www.datacamp.com/community/tutorials/mysql-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034358&utm_targetid=dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1010294&gclid=CjwKCAjw5Kv7BRBSEiwAXGDElc63RDm5lkOeJ2qh-B7Ma1zKwsdujvafZTur-1_iyXBgWI7rBJ7iZxoChT4QAvD_BwE">Data Camp - Getting Started with MySQL in Python</a> 
